This code is for Example 2a when $A = -100(\cos( f_1)\sin(f_2), \sin(f_1)\cos(f_2)))$, where $f_1 = 5\pi \sin(x^2 + y^2), f_2 =5\pi \cos(x^2 + y^2)$

# This section is to display the graph of A and F

In [1]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Generate computation domain (-1,1) x (-1,1)
domain = WorkPlane().MoveTo(-1, -1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
# Generate a triangular mesh of mesh-size 0.01, modify maxh value for resutls with h=0.03 and h=0.05
mesh = Mesh(geo.GenerateMesh(maxh=0.01))

# Define vector field A and FEM space for solving the Neumann problem, fix the local polynomial degree at p=3
a =100
A = CF((-a*cos(5*pi*sin(x**2 + y**2))*sin(5*pi*cos(x**2 + y**2)), -a*sin(5*pi*sin(x**2 + y**2))*cos(5*pi*cos(x**2 + y**2))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

# Define trial-function, test-function, and the variation form 
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

# the solution field
gfm.vec.data = a.mat.Inverse() * f.vec
Draw(gfu, mesh); 

F = A - grad(gfu)
F0x, F1x = F.Diff(x)
F0y, F1y = F.Diff(y)
Draw(A, mesh,vector=True)
Draw(F, mesh,vector=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm for A and F

In [2]:
sqrt(Integrate(A*A, mesh)), sqrt(Integrate(F*F, mesh))

(130.64359041675533, 89.86141987877336)

# This section is to calculate the eigenpairs for H(A)

In [3]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG

start1 = time.time()
a = 100
A = CF((-a*cos(5*pi*sin(x**2 + y**2))*sin(5*pi*cos(x**2 + y**2)), -a*sin(5*pi*sin(x**2 + y**2))*cos(5*pi*cos(x**2 + y**2))))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.01))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
# X.ndof
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

# Use FEAST algorithm for computing eigenvalues
seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 50, center = 153; When h=0.03, radius = 50, center = 153; When h=0.05, radius = 53, center = 186
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=50,
                   center=150,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
# print('Prepare time', start2-start1)
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=50, Center=150+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +197.553+15.451j
SpectralProjNG:   Factorizing at z = +179.389+40.451j
SpectralProjNG:   Factorizing at z = +150.000+50.000j
SpectralProjNG:   Factorizing at z = +120.611+40.451j
SpectralProjNG:   Factorizing at z = +102.447+15.451j
SpectralProjNG:   Factorizing at z = +102.447-15.451j
SpectralProjNG:   Factorizing at z = +120.611-40.451j
SpectralProjNG:   Factorizing at z = +150.000-50.000j
SpectralProjNG:   Factorizing at z = +179.389-40.451j
SpectralProjNG:   Factorizing at z = +197.553-15.451j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [105.46974925 122.46392149 156.53657498 180.32015424 196.59275662
 197.37313798 198.70568967 201.49648963]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [ ]:
# Calculate the degree of freedom 
X.ndof 

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [ ]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

Calculate the phase of $\psi_1$

In [ ]:
eigv = y.MDComponent(0)
yl2 = GridFunction(H1(mesh,order=3))
eigv_phase1 = asin(eigv.imag/eigv.Norm())
Draw(eigv_phase1, mesh)

# This section is to calculate the eigenpairs for H(F)

In [5]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.01))
a = 100
A = CF((-a*cos(5*pi*sin(x**2 + y**2))*sin(5*pi*cos(x**2 + y**2)), -a*sin(5*pi*sin(x**2 + y**2))*cos(5*pi*cos(x**2 + y**2))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object,  radius = 50, center = 153
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=50,
                   center=150,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=50, Center=150+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +197.553+15.451j
SpectralProjNG:   Factorizing at z = +179.389+40.451j
SpectralProjNG:   Factorizing at z = +150.000+50.000j
SpectralProjNG:   Factorizing at z = +120.611+40.451j
SpectralProjNG:   Factorizing at z = +102.447+15.451j
SpectralProjNG:   Factorizing at z = +102.447-15.451j
SpectralProjNG:   Factorizing at z = +120.611-40.451j
SpectralProjNG:   Factorizing at z = +150.000-50.000j
SpectralProjNG:   Factorizing at z = +179.389-40.451j
SpectralProjNG:   Factorizing at z = +197.553-15.451j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [107.83820946 119.21035507 155.90681392 179.86964475 196.61003238
 197.82131645 199.0913813  201.63329406]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [ ]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

Calculate the phase of $\phi_1$

In [ ]:
eigv = y1.MDComponent(0)
yl2 = GridFunction(H1(mesh,order=6))
eigv_phase1 = asin(eigv.imag/eigv.Norm())
Draw(eigv_phase1, mesh)

In [6]:
Draw(y1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene